In [24]:
import torch
import pandas as pd
import os

In [29]:
class bert:
    def __init__(self):
        model = torch.load('./bert_model')
    
    def pred_tweet(self, tweet_data):
        document_bert = ["[CLS] " + str(s) + " [SEP]" for s in tweet_data]
        tokenized_texts = [tokenizer.tokenize(s) for s in document_bert]
        MAX_LEN = 512
        input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
        input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')
        attention_masks = []
        for seq in input_ids:
            seq_mask = [float(i>0) for i in seq]
            attention_masks.append(seq_mask)
        train_inputs = torch.tensor(input_ids)
        train_labels = torch.zeros(len(input_ids)) # 라벨이 없는 트위터 데이터므로 0을 채워넣음, 트위터 개수
        train_masks = torch.tensor(attention_masks)
        train_data = TensorDataset(train_inputs, train_masks, train_labels)
        train_sampler = RandomSampler(train_data)
        train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

        model.eval()
        # 변수 초기화
        pred = []
        # 데이터로더에서 배치만큼 반복하여 가져옴
        for step, batch in enumerate(train_dataloader):
            # 경과 정보 표시
            if step % 100 == 0 and not step == 0:
                elapsed = format_time(time.time() - t0)
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

            # 배치를 GPU에 넣음
            batch = tuple(t.to(device) for t in batch)

            # 배치에서 데이터 추출
            b_input_ids, b_input_mask, b_labels = batch

            # 그래디언트 계산 안함
            with torch.no_grad():     
                # Forward 수행
                outputs = model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask)

            # 로스 구함
            logits = outputs[0]

            # CPU로 데이터 이동
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            #예측된 값을 저장
            pred += np.argmax(logits, axis=1).flatten().tolist()
        return pred

In [30]:
a = bert()

FileNotFoundError: [Errno 2] No such file or directory: './bert_model'

In [19]:
b = [['i love you']]

In [25]:
import nltk
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
## 1. 트윗 전처리

file_dir = './data'
df = pd.DataFrame()

# 디렉토리에서 트윗 파일 불러오기
for file_name in os.listdir(file_dir):
    # 트윗 파일이 아니면 넘김
    #if ('PLTR' not in file_name) and ('palantir' not in file_name):
    if 'stock' in file_name:
        print(f"FIle skipped (not tweet): {file_name}")
        continue

    file_path = os.path.join(file_dir, file_name)
    
    # 트윗 파일 로드 및 병합
    try:
        file = pd.read_csv(file_path, index_col=0)
        if len(file) == 0:
            continue
        df = df.append(file)

    except Exception as error_message:
        print(f'error: {error_message}')
        
# 인덱스 초기화
df = df.reset_index(drop=True)
df = df.rename(columns={'text':'tweet'})
print(df)
def clean_tweet(tweet):
    tweet = str(tweet)
    tweet = tweet.lower() # 소문자로 바꾸기
    tweet = re.sub('https?:\/\/[a-zA-Z0-9@:%._\/+~#=?&;-]*', ' ', tweet) # URL 제거
    tweet = re.sub('\$[a-zA-Z0-9]*', ' ', tweet) # ticker symbol($로 시작하는 주식 관련 심볼) 제거
    tweet = re.sub('\@[a-zA-Z0-9]*', ' ', tweet) # 유저 호출하는 기능(@로 시작) 제거
    tweet = re.sub('[^a-zA-Z\']', ' ', tweet) # 문자가 아닌 것 제거
    tweet = ' '.join( [w for w in tweet.split() if len(w)>1] )
    
    tweet = ' '.join([lemma.lemmatize(x) for x in nltk.wordpunct_tokenize(tweet) if x not in stop_words])
    tweet = [lemma.lemmatize(x, nltk.corpus.reader.wordnet.VERB) for x in nltk.wordpunct_tokenize(tweet) if x not in stop_words]
    return tweet 
lemma = WordNetLemmatizer()
stop_words = stopwords.words("english")
# 트윗을 토큰화시킨 것, 그리고 토큰을 이어붙인 것을 새로운 열에 추가
df["clean_tweet"] = df["tweet"].apply(lambda x:clean_tweet(x))
df["cleaned_tweet"] = df["clean_tweet"].apply(lambda x:' '.join(x))
df = df[df['search_word'] == 'Tesla'] # 주식 이름 선택
tweet_data = df['tweet']

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\haeji\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\haeji\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\haeji\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


error: [Errno 13] Permission denied: './data\\.ipynb_checkpoints'
error: [Errno 13] Permission denied: './data\\aclImdb'
       search_word                     user  \
0         GameStop              Marcus Penn   
1         GameStop  h0bbes. Yeah, that one.   
2         GameStop                Omer Zach   
3         GameStop             Bill English   
4         GameStop                     Mae.   
...            ...                      ...   
102531       Tesla            Martin Burris   
102532       Tesla           UNICORN FOR PM   
102533       Tesla   Blocked by Sawyer Club   
102534       Tesla                  News 10   
102535       Tesla             Dan Burkland   

                                                    tweet  \
0       Yay geeking out! (@ GameStop) http://4sq.com/a...   
1       I'm at GameStop (8336 Agora Pkwy, The Forum, S...   
2       I just ousted \n@braisinhussy\n as the mayor o...   
3       I'm at Gamestop in Trenton, NJ http://gowal.la...   
4       I

In [27]:
tweet_data

8004                       Dads new car tesla model x\n1\n2
8005      What an amazing machine! resolution to GoGreen...
8006      Tesla appears to be suffering from to much #Ne...
8007      Shahed: what if tesla had a New Year sale and ...
8008      Tesla Model S Burns To A Crisp During Supercha...
                                ...                        
102531    To anyone who can read this tweet and is a gun...
102532    Tesla is on . Literally. Again!\nJulia Bagg\n@...
102533    NEWS: Tesla has filed a rule change for grid o...
102534    "Luckily, it didn't happen while we were drivi...
102535    For those paying attention, this also confirms...
Name: tweet, Length: 94532, dtype: object

In [28]:
a.pred_tweet(tweet_data)

TypeError: pred_tweet() takes 1 positional argument but 2 were given